### Capturing word correlation in input data

In [1]:
import numpy as np
onehots = {}
onehots['cat'] = np.array([1,0,0,0])
onehots['the'] = np.array([0,1,0,0])
onehots['dog'] = np.array([0,0,1,0])
onehots['sat'] = np.array([0,0,0,1])

sentence = ['the','cat','sat']
x = onehots[sentence[0]]+onehots[sentence[1]]+onehots[sentence[2]]

print("Sentence Encoding:" + str(x))

Sentence Encoding:[1 1 0 1]


### Predicitng movie reviews

In [2]:
import sys


In [3]:
# reading the reviews and labels into a list
f = open('reviews.txt')
raw_reviews = f.readlines()
f.close()

f = open('labels.txt')
raw_labels = f.readlines()
f.close()

In [4]:
# mapping each review as tokens
tokens = list(map(lambda x:set(x.split(" ")),raw_reviews))

In [5]:
"""
creating a set of vocab taking each word from tokens and if len > 0
we add it to the list of words
"""
vocab = set()
for sent in tokens:
    for word in sent:
        if(len(word)>0):
            vocab.add(word)
vocab = list(vocab)

In [6]:
#enumerating each word to find assign index for each of the words

word2index = {}
for i,word in enumerate(vocab):
    word2index[word]=i

In [7]:
# Converting each sentence to index and then creating set of each token to avoid duplicates

input_dataset = list()
for sent in tokens:
    sent_indices = list()
    for word in sent:
        try:
            sent_indices.append(word2index[word])
        except:
            ""
    input_dataset.append(list(set(sent_indices)))

In [8]:
# Converting labels to 0 and 1s
target_dataset = list()
for labels in raw_labels:
    if labels == 'positive\n':
        target_dataset.append(1)
    else:
        target_dataset.append(0)

In [9]:
import numpy as np
np.random.seed(1)

In [10]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [11]:
alpha, iteration = (0.01,2)
hidden_size = 100

In [12]:
weights_0_1 = 0.2*np.random.random((len(vocab),hidden_size)) - 0.1
weights_1_2 = 0.2*np.random.random((hidden_size,1)) - 0.1

#### Training with 2 layer network

In [14]:
correct,total = (0,0)
for iter in range(iteration):
    for i in range(len(input_dataset)-1000):
        x,y = (input_dataset[i],target_dataset[i])
        layer_1 = sigmoid(np.sum(weights_0_1[x],axis=0))
        layer_2 = sigmoid(np.dot(layer_1,weights_1_2))
        
        layer_2_delta = layer_2 - y
        layer_1_delta = layer_2_delta.dot(weights_1_2.T)
        
        weights_0_1[x] -= layer_1_delta * alpha
        weights_1_2 -= np.outer(layer_1,layer_2_delta)*alpha
        
        if(np.abs(layer_2_delta)<0.5):
            correct += 1
        total += 1
        if(i % 10 == 9):
            progress = str(i/float(len(input_dataset)))
            sys.stdout.write('\rIter:'+str(iter)\
                +' Progress:'+progress[2:4]\
                             +'.'+progress[4:6]\
                +'% Training Accuracy:'\
                + str(correct/float(total)) + '%')
    print()
correct,total = (0,0)
for i in range(len(input_dataset)-1000,len(input_dataset)):
    x = input_dataset[i]
    y = target_dataset[i]
    layer_1 = sigmoid(np.sum(weights_0_1[x],axis=0)) 
    layer_2 = sigmoid(np.dot(layer_1,weights_1_2))
    correct += 1
    total += 1
print("Test Accuracy:" + str(correct / float(total)))

Iter:0 Progress:05.59% Training Accuracy:0.5757142857142857%

Iter:0 Progress:11.63% Training Accuracy:0.6907216494845361%

Iter:0 Progress:17.79% Training Accuracy:0.7271910112359551%

Iter:0 Progress:23.99% Training Accuracy:0.751%

Iter:0 Progress:29.99% Training Accuracy:0.7713333333333333%

...

Iter:0 Progress:40.99% Training Accuracy:0.7946341463414635%

Iter:0 Progress:46.99% Training Accuracy:0.8016170212765957%

Iter:0 Progress:52.75% Training Accuracy:0.8075056861258529%

Iter:0 Progress:58.47% Training Accuracy:0.8116963064295486%

Iter:0 Progress:95.99% Training Accuracy:0.83275%6515214673%
Iter:1 Progress:95.99% Training Accuracy:0.8665%38570535528%
Test Accuracy:1.0


#### Comparing word embeddings

In [18]:
from collections import Counter
import math 

def similar(target='beautiful'):
    target_index = word2index[target]
    scores = Counter()
    for word,index in word2index.items():
        raw_difference = weights_0_1[index] - (weights_0_1[target_index])
        squared_difference = raw_difference * raw_difference
        scores[word] = -math.sqrt(sum(squared_difference))

    return scores.most_common(10)

In [19]:
print(similar('beautiful'))

[('beautiful', -0.0), ('great', -0.7200525239027686), ('appreciated', -0.7336346571486223), ('true', -0.7431760229710972), ('masterpiece', -0.748550344826609), ('ride', -0.7499993307722741), ('innocent', -0.7524499504157217), ('simple', -0.7651353019024564), ('sweet', -0.7683005370533182), ('worlds', -0.7689800163230051)]


In [20]:
print(similar('terrible'))

[('terrible', -0.0), ('badly', -0.7505059803595728), ('avoid', -0.756439386511795), ('boring', -0.7942688893924995), ('annoying', -0.7945404447524166), ('worse', -0.810109328606382), ('horrible', -0.8172600822957408), ('disappointment', -0.8269188772893417), ('fails', -0.8394884990136618), ('lacks', -0.8435591977629718)]
